<a href="https://colab.research.google.com/github/braveenth/Canadian-Weather-AI/blob/main/Canadian_Weather_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Canadian Weather AI
By: Braveenth Rasanayagam

Multi-modal
- Vision
- Text to Voice

Cloud Run Jobs

In [1]:
image_url = "https://weather.gc.ca/data/jet_stream/tempmapwx_e.gif"
print(image_url)
!curl -o case_image.png {image_url}

from IPython.display import Image
Image("/content/jet_stream.png")
Image(url=image_url)

https://weather.gc.ca/data/jet_stream/tempmapwx_e.gif
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  283k  100  283k    0     0   635k      0 --:--:-- --:--:-- --:--:--  636k


In [19]:
import os

try:
    # Attempt to import the google.colab module to see if the program is running in Colab.
    from google.colab import userdata
    print("Running on Google Colab")
    !pip install openai
    !pip install pydub
    from openai import OpenAI
    client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
    !mkdir -p /content/text
    !mkdir -p /content/voice
    !mkdir -p /content/assets
    !curl -o "/content/assets/WeatherNetwork.mp3" "https://storage.googleapis.com/can-weather-ai/assets/WeatherNetwork.mp3"

except ImportError:
    # The ImportError exception will be raised if the google.colab module is not found,
    # indicating that the program is not running inside Google Colab.
    print("Running outside of Google Colab")
    client = OpenAI(api_key=(os.environ.get('OPENAI_API_KEY')))

from openai import OpenAI
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

Running on Google Colab
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 64.1M  100 64.1M    0     0  44.5M      0  0:00:01  0:00:01 --:--:-- 44.5M


In [10]:
canadianMetric = {}

canadianMetric[0] = "not Canadian"
canadianMetric[1] = "somewhat Canadian"
canadianMetric[2] = "slightly Canadian"
canadianMetric[3] = "moderately Canadian"
canadianMetric[4] = "very Canadian"
canadianMetric[5] = "as Canadian as possible"

In [5]:
gpt_mode = "gpt-4-vision-preview"
imagePrompt = "You are a Canadian weatherperson. Present this jet stream information to an audience. Make it sound very Canadian. It should sound as Canadian as absolutely possible."
image_url = "https://weather.gc.ca/data/jet_stream/tempmapwx_e.gif"

response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": imagePrompt},
        {
          "type": "image_url",
          "image_url": {
            "url": image_url,
          },
        },
      ],
    }
  ],
  max_tokens=600,
  temperature=0.7,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Alrighty folks, grab yourself a double-double and let's take a look at the ole jet stream, eh? Now, we've got some classic Canadian weather on the map today, and by that, I mean it's all over the place, just like the hockey standings, am I right?\n\nFirst off, we've got a big ol' high-pressure system sitting like a friendly moose over the Prairies, eh, with a temperature colder than a polar bear's toenails, especially in the blue areas. So you might want to keep your toque handy and maybe throw on an extra flannel layer if you're heading out for a rip.\n\nNow, those red arrows are showing us the path of the jet stream, swooping down faster than a loon dives for a fish. This is pushing some chilly air right from the true north strong and free down into our neighbours to the south. So don't be surprised if you hear our American buddies talkin' about a Canadian chill, eh?\n\nOver in the Atlantic pr

In [14]:
weatherScript = response.choices[0].message.content
print(weatherScript)

Alrighty folks, grab yourself a double-double and let's take a look at the ole jet stream, eh? Now, we've got some classic Canadian weather on the map today, and by that, I mean it's all over the place, just like the hockey standings, am I right?

First off, we've got a big ol' high-pressure system sitting like a friendly moose over the Prairies, eh, with a temperature colder than a polar bear's toenails, especially in the blue areas. So you might want to keep your toque handy and maybe throw on an extra flannel layer if you're heading out for a rip.

Now, those red arrows are showing us the path of the jet stream, swooping down faster than a loon dives for a fish. This is pushing some chilly air right from the true north strong and free down into our neighbours to the south. So don't be surprised if you hear our American buddies talkin' about a Canadian chill, eh?

Over in the Atlantic provinces, watch out for some snow, as indicated by the white regions on the map. Perfect for firing

In [15]:
# Save the weatherScript as a file
from datetime import datetime

date = datetime.now().strftime("%Y-%m-%d")
count = 0

while os.path.exists(f"/content/text/ai-canadian-jetstream-{date}_{count}.txt"):
    count += 1
file = open(f"/content/text/ai-canadian-jetstream-{date}_{count}.txt", "w")
file.write(weatherScript)
file.close()

In [16]:
from openai import OpenAI

try:
    # Attempt to import the google.colab module to see if the program is running in Colab.
    from google.colab import userdata
    print("Running on Google Colab")
    audioClient = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

except ImportError:
    # The ImportError exception will be raised if the google.colab module is not found,
    # indicating that the program is not running inside Google Colab.
    import os
    print("Running outside of Google Colab")
    audioClient = OpenAI(api_key=(os.environ.get('OPENAI_API_KEY')))

audioResponse = audioClient.audio.speech.create(
    model="tts-1",
    voice="onyx",
    input=weatherScript,
)

filename = f"weatherperson-output-{date}.mp3"

Running on Google Colab


In [17]:
audioResponse.stream_to_file(filename)

<ipython-input-17-02f298be9c94>:1: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  audioResponse.stream_to_file(filename)


In [20]:
from pydub import AudioSegment

# Export the result
weatherPersonFileCount = 0
while os.path.exists(f"/content/weatherperson-output-{date}_{weatherPersonFileCount}.wav"):
    weatherPersonFileCount += 1

weatherpersonFile = f"/content/{filename}-{count}"
audioResponse.stream_to_file(f"/content/weatherperson-output-{date}_{weatherPersonFileCount}.wav")

# Obtain Audio Files
audio_bg_music = AudioSegment.from_file("/content/assets/WeatherNetwork.mp3", format="mp3")
audio_weatherperson = AudioSegment.from_file(f"/content/weatherperson-output-{date}_{weatherPersonFileCount}.wav", format="mp3")

# Define start and end times in milliseconds
# Start at 17:22
# End at 17:3
start_time = 1042000  # Start at 10 seconds
end_time = start_time + 120000

# Slice the audio segment to the desired part
specific_part = audio_bg_music[start_time:end_time]

# Lower the volume by 10 dB
specific_part_quieter = specific_part - 5

# Overlay the audio files
# Here, overlay_audio will start at 0 milliseconds into base_audio
combined = specific_part_quieter.overlay(audio_weatherperson, position=0)

# Export the result
count = 0
while os.path.exists(f"/content/voice/ai-canadian-jetstream-{date}_{count}.wav"):
    count += 1
combined.export(f"/content/voice/ai-canadian-jetstream-{date}_{count}.mp3", format="mp3")


<ipython-input-20-abb7955a0c2c>:9: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  audioResponse.stream_to_file(f"/content/weatherperson-output-{date}_{weatherPersonFileCount}.wav")


<_io.BufferedRandom name='/content/voice/ai-canadian-jetstream-2024-03-19_0.mp3'>